# SUMO_DNDC

A surrogate model to LandscapeDNDC for faster inference... 😎

The plan is to model a time-series via a LSTM. Trainging data is provided by LandscapeDNDC model runs.

<figure>
    <img src="../images/sketch.png" alt="Sketch" width="400"/>
    <figcaption>Fig. 1: A sketch of the model setup</figcaption>
</figure>

## Embeddings

We encode the site state in one embedding layer. The current format is:

### General site info  

| id   | variable | min     | max     |      
|------|----------|---------|---------|
| 0    | site lat | NA      | NA      |
| 1    | site lon | NA      | NA      |
| 2    | site ele | 0       | NA      |


### Airchemistry info  

| id   | variable | min     | max     |      
|------|----------|---------|---------|
| 3    | nh4      | 0       | NA      |
| 4    | no3      | 0       | NA      |
| 5    | ch4      | 0       | NA      |
| 6    | co2      | 0       | NA      |
| 7    | o3       | 0       | NA      |
| 8    | nh3      | 0       | NA      |
| 9    | no2      | 0       | NA      |
| 10   | no       | 0       | NA      |


### Soil properties

Soil variables are aggregated/ expanded to 10cm depth buckets.
The top 50cm of any profile is used (depth and split are used
to preprocess soil strata to fixed 10cm bucket format. SKS is
omitted.

| id   | variable | min     | max     |      
|------|----------|---------|---------|
| 11   | bd (0)   | 0       | NA      |
| 12   | clay (0) | 0       | NA      |
| 13   | corg (0) | 0       | NA      |
| 14   | norg (0) | 0       | NA      |
| 15   | ph (0)   | 0       | NA      |
| 16   | sand (0) | 0       | NA      |
| 17   | scel (0) | 0       | NA      |
| 18   | silt (0) | 0       | NA      |
| 19   | wcmin (0)| 0       | NA      |
| 20   | wcmax (0)| 0       | NA      |
| ...  | ...      | ...     | ...     |
| 60   | wcmax (4)| 0       | NA      |

### Other

Not yet considered... Possibly site history?

----

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, '..') # Add directory above current directory to path

In [2]:
# general imports

In [3]:
from sumo_dndc.parser import Parser, InFile, OutFile

In [4]:
airchem = Parser(InFile.AIRCHEM, "../data/DE_fendt/DE_fendt_airchem.txt")
airchem

Creating Parser:<class 'sumo_dndc.parser.AirchemParser'>
Parsing TxtFile ../data/DE_fendt/DE_fendt_airchem.txt


Parser: InFile.AIRCHEM, ../data/DE_fendt/DE_fendt_airchem.txt
Data excerpt:
   nh4  no3  ch4    co2   o3  nh3  no2   no
0  1.4  0.9  0.0  400.0  0.0  0.0  0.0  0.0

In [5]:
climate = Parser(InFile.CLIMATE)
climate.parse("../data/DE_fendt/DE_fendt_climate.txt")
climate

Creating Parser:<class 'sumo_dndc.parser.ClimateParser'>
Parsing TxtFile ../data/DE_fendt/DE_fendt_climate.txt


Parser: InFile.CLIMATE, ../data/DE_fendt/DE_fendt_climate.txt
Data excerpt:
          *  prec      tavg  tmax   tmin       grad         rh       press  \
0  01.01.10   0.8 -1.558333 -0.12  -3.10  23.501792  89.145833  948.694444   
1  02.01.10   0.6 -1.248194  0.00  -3.04  26.181847  83.873333  948.902778   
2  03.01.10   0.0 -4.909329  1.13  -9.49  19.080529  82.456224  950.587413   
3  04.01.10   0.0 -8.494789 -4.16 -12.14  29.938732  85.451761  946.394366   
4  05.01.10   0.0 -8.978889  0.28 -14.18  12.868179  85.121875  942.763889   

       wind  
0  1.396806  
1  1.837500  
2  1.143014  
3  0.810704  
4  0.786667  

In [6]:
site = Parser(InFile.SITE, "../data/DE_fendt/DE_fendt_site.xml")

Creating Parser:<class 'sumo_dndc.parser.SiteParser'>


In [7]:
site

Parser: InFile.SITE, ../data/DE_fendt/DE_fendt_site.xml
Data excerpt:
<?xml version="1.0" encoding="utf8"?>
<soil>
	<general humus="MULL" litterheight="" soil="LOAM" usehistory="grassland"/>
	<layers>
		<layer bd="0.74" clay="0.272" corg="0.0679" depth="50" norg="0.0066" ph="5.1" sand="0.305" scel="0.00" silt="0.403" sks="0.020" split="3" wcmax="500" wcmin="235"/>
		<layer bd="1.10" clay="0.252" corg="0.0435" depth="150" norg="0.0048" ph="6.6" sand="0.355" scel="0.038" silt="0.403" sks="0.020" split="8" wcmax="460" wcmin="235"/>

In [8]:
airchem

Parser: InFile.AIRCHEM, ../data/DE_fendt/DE_fendt_airchem.txt
Data excerpt:
   nh4  no3  ch4    co2   o3  nh3  no2   no
0  1.4  0.9  0.0  400.0  0.0  0.0  0.0  0.0